<a href="https://colab.research.google.com/github/sarang-manohar/hogwarts/blob/main/academia/sig788/code/task_4_1_P_Sarang_Manohar(s223504903)_Part2_v1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install fiftyone
!pip install azure-cognitiveservices-vision-customvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.5/564.5 KB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.8/37.8 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.1/492.1 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

In [3]:
import json

with open('creds.txt') as f:
    contents = f.read()
    creds = json.loads(contents)

In [4]:
credentials = ApiKeyCredentials(in_headers={"Training-key": creds['training_key']})
trainer = CustomVisionTrainingClient(creds['ENDPOINT'], credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": creds['prediction_key']})
predictor = CustomVisionPredictionClient(creds['ENDPOINT'], prediction_credentials)

In [5]:
with open('project_id.txt') as f:
    project_id = f.read()
project_id

'e9c32e6a-a241-401c-bc7f-e859fb047664'

In [6]:
tag = trainer.create_tag(project_id, "bowl")

In [17]:
# Import fiftyone
import fiftyone.zoo as fo

# Import data dataset 
def load_dataset(max_samples, seed):
    dataset_train = fo.load_zoo_dataset(
        "coco-2017",
        split="train",
        label_types=["detections"],
        classes=["bowl"],
        max_samples=max_samples,
        seed=seed,
        dataset_name="oi-bowl-train" + str(seed),
    )
    return dataset_train

In [18]:
dataset_train = load_dataset(max_samples=1000, seed=3)

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/coco-2017/train' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_train2017.json'


Sufficient images already downloaded


INFO:fiftyone.utils.coco:Sufficient images already downloaded


...

INFO:fiftyone.zoo.datasets:Loading 'coco-2017' split 'train'


 100% |███████████████| 1000/1000 [12.2s elapsed, 0s remaining, 59.7 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 1000/1000 [12.2s elapsed, 0s remaining, 59.7 samples/s]      


Dataset 'oi-bowl-train3' created


INFO:fiftyone.zoo.datasets:Dataset 'oi-bowl-train3' created


In [19]:
import fiftyone as fo
from fiftyone.types.dataset_types import COCODetectionDataset

# Get the path to the annotations file
annFile = '/root/fiftyone/coco-2017/train/labels.json'

from pycocotools.coco import COCO

# Load the annotations file
coco = COCO(annFile)

# Get all the annotations in the dataset
annIds = coco.getAnnIds()
anns = coco.loadAnns(annIds)

# Convert each annotation to the left, top, width, height format
for ann in anns:
    x, y, w, h = ann['bbox']
    left = x
    top = y
    width = w
    height = h
    ann['bbox'] = [left, top, width, height]

import json
# Save the modified annotations file
with open('/root/fiftyone/coco-2017/train/updated_annotation.json', 'w') as f:
    json.dump(coco.dataset, f)

loading annotations into memory...
Done (t=0.14s)
creating index...
index created!


In [20]:
with open('/root/fiftyone/coco-2017/train/updated_annotation.json') as f:
    contents = f.read()
    annotations = json.loads(contents)
    
import pandas as pd

tmp = pd.DataFrame(annotations['images'])
tmp = tmp[['file_name','height','width','id']].copy()
tmp2 = pd.DataFrame(annotations['annotations'])
tmp2 = tmp2[['image_id','bbox','area','category_id']].copy()
tmp2.rename(columns={'image_id':'id'}, inplace=True)

#merge dataframes
merged_data = pd.merge(tmp,tmp2, how='inner')
tmp3 = pd.DataFrame(annotations['categories'])
tmp3 = tmp3[tmp3['name']=='bowl'].copy()
tmp3.rename(columns={'id':'category_id'}, inplace=True)
merged_data = pd.merge(merged_data,tmp3, how='inner')
merged_data[['Left','Top','Width','Height']] = pd.DataFrame(merged_data.bbox.tolist(), index= merged_data.index)
merged_data['Left'] = merged_data['Left'] / merged_data['width'] 
merged_data['Top'] = merged_data['Top'] / merged_data['height'] 
merged_data['Width'] = merged_data['Width'] / merged_data['width'] 
merged_data['Height'] = merged_data['Height'] / merged_data['height'] 
data_to_upload = merged_data[['file_name','Left','Top','Width','Height']] 
dict = data_to_upload.set_index('file_name').T.to_dict('list')

In [21]:
import shutil

data_dir = os.path.join('/root/fiftyone/coco-2017/train/data')
temp_dir = os.path.join('/root/fiftyone/coco-2017/train/temp')
if not os.path.exists(temp_dir):
    os.mkdir(temp_dir)

data = pd.DataFrame(dict)

In [23]:
batch_size = 50
for i in range(0, data.shape[1], batch_size):
    if i != 0:
            print(i," files uploaded so far")
    # Get the next batch of files
    batch_df = data.iloc[:,i:i+batch_size]
    tagged_images_with_regions = []
    file_list = []

    for file_name in batch_df.to_dict().keys():
        x,y,w,h = batch_df[file_name]
        regions = [ Region(tag_id=tag.id, left=x,top=y,width=w,height=h) ]
        file_list.append(file_name)

    for file in file_list:
        shutil.copy(os.path.join(data_dir, file), os.path.join(temp_dir, file))
        with open(os.path.join(temp_dir, file ), mode="rb") as image_contents:
            tagged_images_with_regions.append(ImageFileCreateEntry(name=file, contents=image_contents.read(), regions=regions))

        upload_result = trainer.create_images_from_files(project_id, ImageFileCreateBatch(images=tagged_images_with_regions))
    
    for file in file_list:
        os.remove(os.path.join(temp_dir, file))
        


50  files uploaded so far
100  files uploaded so far
150  files uploaded so far
200  files uploaded so far
250  files uploaded so far
300  files uploaded so far
350  files uploaded so far
400  files uploaded so far
450  files uploaded so far
500  files uploaded so far
550  files uploaded so far
600  files uploaded so far


KeyboardInterrupt: ignored